## explore_expanded.ipynb

A notebook for exploring expanded events in the Unizin Data Platform

In [ ]:
# standard libraries
import json, os

# third-party libraries
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas as pd
import matplotlib.pyplot as plt

pd.set_option('display.max_rows', None)

First, set up the BigQuery client.
See https://cloud.google.com/bigquery/docs/authentication/service-account-file#python

In [ ]:
key_path = os.path.join(os.path.abspath('.'), 'unizin-hackathon-updated.json')

cred = service_account.Credentials.from_service_account_file(
    key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"]
)

client = bigquery.Client(credentials=cred, project=cred.project_id)

Start executing queries!

### Exploration of expanded events

Fetch events from expanded table using the client.

In [ ]:
events_query = '''
    select
        id,
        event_time,
        type,
        action,
        actor,
        target,
        object.id as object_id
    from session1_datamarts.expanded
    where event_time >= '2021-01-01';
'''

expanded_event_data = client.query(events_query)
event_df = expanded_event_data.to_dataframe()

Begin explanation/analysis!

In [ ]:
event_df.head(2)

In [ ]:
len(event_df)

In [ ]:
event_df.columns

In [ ]:
event_df['type'].unique()

In [ ]:
event_df_type = event_df[['id', 'type']].copy()
event_df_type.groupby(['type']).count()

In [ ]:
nav_event_df = event_df.loc[event_df['type'] == 'NavigationEvent'].copy()
len(nav_event_df)

In [ ]:
# Transform object_id
nav_event_df['object_id_wo_canvas'] = nav_event_df['object_id'].map(lambda x: ':'.join(x.split(':')[-2:]))
nav_event_df.head(2)

In [ ]:
nav_event_df_less = nav_event_df[['id', 'object_id']].copy()
nav_count_df = nav_event_df_less.groupby(['object_id']).count()
nav_count_df = nav_count_df.rename(columns={'id': 'nav_count'})
nav_count_df = nav_count_df.sort_values(by=['nav_count'], ascending=False)
nav_count_df[:100]

### Exploration of course_offering_stats

Fetch 2020 from course_offering_status data mart.

In [ ]:
course_offering_stats_query = '''
    select
        lms_course_offering_id,
        udp_course_offering_id,
        course_offering_subject,
        learning_environment_status,
        num_learner_events,
        academic_term_name
    from session1_datamarts.course_offering_stats
    where academic_term_year=2020;
'''

course_stats_data = client.query(course_offering_stats_query)
course_stats_df = course_stats_data.to_dataframe()

Begin exploration/analysis.

In [ ]:
len(course_stats_df)

In [ ]:
course_stats_df.head(500)

In [ ]:
course_stats_df['course_offering_subject'].unique()

In [ ]:
course_stats_fall_df = course_stats_df.loc[course_stats_df['academic_term_name'] == 'Fall 2020'].copy()

In [ ]:
def published(value):
    if value == 'available':
        return True
    return False

course_stats_fall_df['published'] = course_stats_df['learning_environment_status'].map(published)
course_stats_fall_df